![](../screenshot/swin0.png)

- 같은 X축에 있으면 상대적 거리가 0, 아래로 떨어지면 1, 위로 떨어지면 -1
- 같은 y축에 있으면 상대적 거리가 0, 오른쪽으로 떨어지면 -1, 왼쪽으로 떨어지면 1

In [5]:
####### 10.10 X, Y의 상대적 위치 편향 계산 (①,② 계산과정)
import torch


window_size     = 2
coords_h        = torch.arange(window_size) # [0,1]
coords_w        = torch.arange(window_size) # [0,1]
# torch.meshgrid로 cords_h(i)배열값과 cords_w(j)배열값으로 사각형 격자(ij)를 만들어 사각형의 좌표에 해당하는 배열 반환
# torch.stack을 사용하여 (X, Y)쌍 배열을 만듦
coords          = torch.stack(torch.meshgrid([coords_h, coords_w], indexing="ij"))
''' tensor([[[0, 0],
            [1, 1]],

            [[0, 1],
            [0, 1]]]) '''

# X, Y축에 대한 위치 인덱스 생성 (첫번째차원(y)를 제외하고 평탄화)
coords_flatten  = torch.flatten(coords, 1)
'''tensor([[0, 0, 1, 1],
        [0, 1, 0, 1]])'''


# 각 위치 색인의 차이 계산
relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
'''[:, :, None] : tensor([[[0],
                            [0],
                            [1],
                            [1]],  <- x축 상대 좌표 [2,4,1]

                            [[0],
                            [1],
                            [0],
                            [1]]])'''
'''[:, None, :] : tensor([[[0, 0, 1, 1]],   <- y축 상대좌표 * -1 [2,1,4]

                        [[0, 1, 0, 1]]])'''
### X, Y에 대한 위치 행렬
print(relative_coords)
print(relative_coords.shape)

tensor([[[ 0,  0, -1, -1],
         [ 0,  0, -1, -1],
         [ 1,  1,  0,  0],
         [ 1,  1,  0,  0]],

        [[ 0, -1,  0, -1],
         [ 1,  0,  1,  0],
         [ 0, -1,  0, -1],
         [ 1,  0,  1,  0]]])
torch.Size([2, 4, 4])


In [7]:
####### 10.11 보정된 X, Y축에 대한 위치 행렬 (③,④)
x_coords = relative_coords[0, :, :]
y_coords = relative_coords[1, :, :]

# 인덱스화 위해 0이상으로 만드는 작업
x_coords += window_size - 1  # X축에 대한 ③번 연산 과정
y_coords += window_size - 1  # Y축에 대한 ③번 연산 과정
x_coords *= 2 * window_size - 1  # ④번 연산 과정
print(f"X축에 대한 행렬:\n{x_coords}\n")
print(f"Y축에 대한 행렬:\n{y_coords}\n")

relative_position_index = x_coords + y_coords  # ⑤번 연산 과정
print(f"X, Y축에 대한 위치 행렬:\n{relative_position_index}")

X축에 대한 행렬:
tensor([[3, 3, 0, 0],
        [3, 3, 0, 0],
        [6, 6, 3, 3],
        [6, 6, 3, 3]])

Y축에 대한 행렬:
tensor([[1, 0, 1, 0],
        [2, 1, 2, 1],
        [1, 0, 1, 0],
        [2, 1, 2, 1]])

X, Y축에 대한 위치 행렬:
tensor([[4, 3, 1, 0],
        [5, 4, 2, 1],
        [7, 6, 4, 3],
        [8, 7, 5, 4]])


In [10]:
####### 10.12 X, Y축에 대한 상대적 위치 좌표 반환
num_heads = 1   # MSA 계산할 때 사용된 헤드 수
# B hat 행렬
relative_position_bias_table = torch.Tensor(
    torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_heads)
)

# 인덱스에 해당하는 부분 가져오기
relative_position_bias = relative_position_bias_table[relative_position_index.view(-1)]
# 가져온 값 M^2 * M^2 꼴로 변환 (B행렬) (M^2: 윈도우 내부의 패치 개수)
relative_position_bias = relative_position_bias.view(
    window_size * window_size, window_size * window_size, -1
)

print(relative_position_bias.shape)

torch.Size([4, 4, 1])
